In [ ]:
import json
from shapely.geometry import shape, MultiPolygon, Polygon, mapping
from shapely.ops import unary_union
from shapely.validation import explain_validity
import pandas as pd
import geopandas as gpd
from shapely import wkt
from branca.colormap import linear
from branca.colormap import LinearColormap
import folium

In [ ]:
data = pd.read_csv('merged_file.csv')

data['geometry'] = data['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(data, geometry='geometry')

gdf.to_file('merged.geojson', driver='GeoJSON')

In [ ]:
import json
from shapely.geometry import shape, MultiPolygon, Polygon, mapping
from shapely.ops import unary_union
from shapely.validation import explain_validity

def validate_and_fix(geom):
    if not geom.is_valid:
        print("Invalid geometry:", explain_validity(geom))
        return geom.buffer(0)
    return geom

def simplify(geom):
    return geom.simplify(0.1, preserve_topology=True)

with open('merged.geojson') as f1:
    data1 = json.load(f1)

with open('Boundaries - TIF.geojson') as f2:
    data2 = json.load(f2)

geoms1 = [validate_and_fix(simplify(shape(feature['geometry']))) for feature in data1['features']]
geoms2 = [validate_and_fix(simplify(shape(feature['geometry']))) for feature in data2['features']]

output_features = []

for geom1, prop1 in zip(geoms1, data1['features']):
    intersecting_parts = []
    for geom2 in geoms2:
        if geom1.intersects(geom2):
            intersection = geom1.intersection(geom2)
            if isinstance(intersection, MultiPolygon):
                intersecting_parts.extend(list(intersection.geoms))
            elif isinstance(intersection, Polygon):
                intersecting_parts.append(intersection)

    non_intersecting_parts = geom1
    for part in intersecting_parts:
        non_intersecting_parts = non_intersecting_parts.difference(part)

    if isinstance(non_intersecting_parts, MultiPolygon):
        for part in non_intersecting_parts.geoms:
            ratio = part.area / geom1.area
            output_features.append({
                'type': 'Feature',
                'geometry': mapping(part),
                'properties': {**prop1['properties'], 'ratio_to_original': ratio},
            })
    elif isinstance(non_intersecting_parts, Polygon):
        ratio = non_intersecting_parts.area / geom1.area
        output_features.append({
            'type': 'Feature',
            'geometry': mapping(non_intersecting_parts),
            'properties': {**prop1['properties'], 'ratio_to_original': ratio},
        })

output_geojson = {
    'type': 'FeatureCollection',
    'features': output_features
}

with open('non_intersecting_geojson_file.geojson', 'w') as output_file:
    json.dump(output_geojson, output_file)


In [ ]:
with open("non_intersecting_geojson_file.geojson") as f:
    census_txt = f.read()

census_tracts = json.loads(census_txt)

In [ ]:
filtered_data = data.dropna(subset=["Median_Household_Income"])
min_income = filtered_data["Median_Household_Income"].min()
max_income = filtered_data["Median_Household_Income"].max()
colormap = linear.RdYlGn_11.scale(min_income, max_income)
colormap

In [ ]:
pop_dict = filtered_data.set_index("name10")["Median_Household_Income"]

In [ ]:
name10 = census_tracts['features'][0]["properties"]["name10"]
income = pop_dict.get(name10)
income

68196.0

In [ ]:
m = folium.Map([43, -100], zoom_start=7)

popup = folium.GeoJsonPopup(fields=["namelsad10","Median_Household_Income","ratio_to_original"])

folium.GeoJson(census_tracts,style_function=lambda feature: {
        "fillColor": colormap(pop_dict[feature["properties"]["name10"]]) if feature["properties"]["name10"] in pop_dict else 'gray',
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.8,
    },
    popup=popup).add_to(m)

colormap.caption = "Income Color Scale"
colormap.add_to(m)
m